In [35]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, average_precision_score


In [36]:

df=pd.read_csv('processed.csv')
df.head()

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,claim sum,claim to total ratio
0,328,48,521585,854,2,1,1000,1406.91,0,466132,...,71610,6510,13020,52080,10,1,2004,1,71610,1.0
1,228,42,342868,579,1,1,2000,1197.22,5000000,468176,...,5070,780,780,3510,8,12,2007,1,5070,1.0
2,134,29,687698,373,2,0,2000,1413.14,5000000,430632,...,34650,7700,3850,23100,4,30,2007,0,34650,1.0
3,256,41,227811,19,0,1,2000,1415.74,6000000,608117,...,63400,6340,6340,50720,3,34,2014,1,63400,1.0
4,256,39,104594,588,2,1,1000,1351.10,0,478456,...,64100,6410,6410,51280,10,3,2003,1,64100,1.0


In [ ]:
#spliting the data set into x and y
x=df.drop(columns=['policy_number','claim to total ratio','claim sum','auto_make','fraud_reported'],axis=1)
y=df['fraud_reported']

In [38]:
print(x)

     months_as_customer  age  policy_bind_date  policy_state  policy_csl  \
0                   328   48               854             2           1   
1                   228   42               579             1           1   
2                   134   29               373             2           0   
3                   256   41                19             0           1   
4                   256   39               588             2           1   
..                  ...  ...               ...           ...         ...   
904                   3   38                61             2           2   
905                 285   41               822             0           0   
906                 130   34               465             2           1   
907                 458   62               753             0           2   
908                 456   60               257             2           1   

     policy_deductable  policy_annual_premium  umbrella_limit  insured_zip  \
0        

In [ ]:
#spliting for training and testing
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.2)

In [40]:
print(x_train.shape,x_test.shape)

(727, 36) (182, 36)


In [41]:
from sklearn.model_selection import StratifiedKFold,cross_val_score


In [ ]:

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from xgboost import XGBClassifier

In [ ]:
#cross validation score for RF
cross_val_score(RandomForestClassifier(n_estimators=60),x,y,cv=10)

array([0.7032967 , 0.76923077, 0.69230769, 0.81318681, 0.79120879,
       0.73626374, 0.79120879, 0.79120879, 0.8021978 , 0.8       ])

In [ ]:
#cross validation score for KNN
cross_val_score(KNeighborsClassifier(),x,y,cv=10)

array([0.71428571, 0.71428571, 0.65934066, 0.72527473, 0.67032967,
       0.67032967, 0.71428571, 0.69230769, 0.67032967, 0.7       ])

In [ ]:
#cross validation score for SVC
cross_val_score(SVC(),x,y,cv=10)

array([0.72527473, 0.73626374, 0.73626374, 0.73626374, 0.72527473,
       0.73626374, 0.73626374, 0.73626374, 0.73626374, 0.73333333])

In [ ]:
#cross validation score for XGB
cross_val_score(XGBClassifier(),x,y,cv=10)

array([0.79120879, 0.82417582, 0.71428571, 0.9010989 , 0.79120879,
       0.8021978 , 0.84615385, 0.84615385, 0.87912088, 0.81111111])

In [130]:
model=XGBClassifier( n_estimators=100,
    learning_rate=0.15,
    max_depth=4)


In [ ]:
#cross validating the data set for limiting data over fitting
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

cv_scores = cross_val_score(model, x_train, y_train, cv=cv, scoring='accuracy')
print("Cross-validation scores:", cv_scores)
print("Mean CV accuracy:", cv_scores.mean())


Cross-validation scores: [0.75342466 0.82191781 0.82191781 0.75342466 0.79452055 0.82191781
 0.83561644 0.81944444 0.88888889 0.83333333]
Mean CV accuracy: 0.8144406392694064


In [132]:
model.fit(x_train,y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [133]:
x_train_prediction=model.predict(x_train)
training_data_accuracy=accuracy_score(y_train,x_train_prediction)
print('accuracy on the training data',training_data_accuracy)
training_precision = precision_score(y_train,x_train_prediction)
print('precision on the training data',training_precision)
recall = recall_score(y_train,x_train_prediction)
print('recall on the training data',recall)
f1 = f1_score(y_train,x_train_prediction)
print('f1 score on the training data',f1)
conf_matrix = confusion_matrix(y_train,x_train_prediction)
print(conf_matrix)

accuracy on the training data 1.0
precision on the training data 1.0
recall on the training data 1.0
f1 score on the training data 1.0
[[540   0]
 [  0 187]]


In [134]:
x_test_prediction=model.predict(x_test)
test_data_accuracy=accuracy_score(y_test,x_test_prediction)
print('accuracy on test data:',test_data_accuracy)
test_precision = precision_score(y_test,x_test_prediction)
print('precision on the test data',test_precision)
recall_test = recall_score(y_test,x_test_prediction)
print('recall on the test data',recall_test)
f1_test = f1_score(y_test,x_test_prediction)
print('f1 score on the test data',f1_test)
conf_matrix_test = confusion_matrix(y_test,x_test_prediction)
print('confusion mtx:',conf_matrix_test)

accuracy on test data: 0.8461538461538461
precision on the test data 0.76
recall on the test data 0.7037037037037037
f1 score on the test data 0.7307692307692307
confusion mtx: [[116  12]
 [ 16  38]]


In [ ]:
#import joblib

In [ ]:
#joblib.dump(model, "model.pkl")

['model.pkl']